<a id='top'></a>

# CSCI 3022: Intro to Data Science - Summer 2019 Practicum 2
***

This practicum is due on Canvas by **11:59 PM on Thursday July 25**. Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  

**Here are the rules:** 

1. All work, code and analysis, must be your own. 
1. You may use your course notes, posted lecture slides, textbooks, in-class notebooks, and homework solutions as resources.  You may also search online for answers to general knowledge questions like the form of a probability distribution function or how to perform a particular operation in Python/Pandas. 
1. This is meant to be like a coding portion of your midterm exam. So, the instructional team will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
1. If something is left open-ended, it is because we want to see how you approach the kinds of problems you will encounter in the wild, where it will not always be clear what sort of tests/methods should be applied. Feel free to ask clarifying questions though.
2. You may **NOT** post to message boards or other online resources asking for help.
3. You may **NOT** copy-paste solutions *from anywhere*.
4. You may **NOT** collaborate with classmates or anyone else.
5. In short, **your work must be your own**. It really is that simple.

Violation of the above rules will result in an immediate academic sanction (*at the very least*, you will receive a 0 on this practicum or an F in the course, depending on severity), and a trip to the Honor Code Council.

**By submitting this assignment, you agree to abide by the rules given above.**

***

**Name**:  Luna McBride

***


**NOTES**: 

- You may not use late days on the practicums nor can you drop your practicum grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a Practicum clarifications thread. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.

---
**Shortcuts:**  [Problem 1](#p1) | [Problem 2](#p2) | [Bottom](#bot)

---

In [358]:
# import whatever packages you think you will need here!
import numpy as np #For the use of arrays for holding and comparisons
import pandas as pd #For the dataframes that hold the data
from scipy import stats #For the f statistic
import math #For functions such as exp() ie e
import statsmodels.api as sm  #For the models that give us all the betas

<br>

---
<a id='p1'></a>
[Back to top](#top)

### [70 points] Problem 1: Multiple Linear Regression to Explain House Hauntings

<img src="https://s-media-cache-ak0.pinimg.com/originals/09/72/01/09720128cff5de4d4af038cd3fcf7f69.jpg" style="width: 300px;"/>

In an effort to control the skyrocketing prices of real estate in the Colorado Front Range, Governor Polis implemented a cutting edge new intervention. This new program oversaw the introduction of ghosts back into their natural ecosystem, after the ghost population seriously dwindled in recent decades due to overhaunting. However, an unfortunate miscalculation has led to haunted houses becoming a very serious problem in Colorado. Modern problems require modern solutions, so Governor Polis has hired you and the famous hedgehog data scientist/part-time ghostbuster Amy to determine what features of a house may be used to best predict a `haunted` score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted).

You decide to use multiple linear regression to understand and predict what factors lead to increased haunted house hazard. You collected a data set from Haunted Zillow, the lesser-known database of haunted house prices and attributes. The data cover a variety of potential features, and you'll find this data in the file `houses.csv`. 

**Response**: 

- $\texttt{haunted}$: a haunting score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted)

**Features**: 

- $\texttt{age}$: age of the house, in years
- $\texttt{area}$: square footage of interior of house
- $\texttt{bathrooms}$: number of bathrooms
- $\texttt{distance metro}$: distance to the nearest major metropolitan area (in miles)
- $\texttt{distance cemetery}$: distance to the nearest cemetery (in miles)
- $\texttt{cats}$: the number of cats within a one-block radius of the house
- $\texttt{howls}$: the number of wolf howls heard on an average night in the house's neighborhood
- $\texttt{clouds}$: what percentage of the sky was covered by clouds (fraction, 0-1)
- $\texttt{precipitation}$: amount of precipitation in the past 72 hours (inches)
- $\texttt{misery index}$: an economic indicator for how miserable the average United States citizen is, based on the unemployment rate and the inflation rate. More [here](https://www.stuffyoushouldknow.com/podcasts/whats-the-misery-index.htm) and [here](https://en.wikipedia.org/wiki/Misery_index_(economics)). Higher values correspond to more miserable citizens.
- $\texttt{ice cream sold}$: the number of units of ice cream sold at the farmer's market the week the house was most recently sold

**Part A**: Read the data from `houses.csv` into a Pandas DataFrame.  Note that since we will be doing a multiple linear regression we will need all of the features, so you should drop any row in the DataFrame that is missing data. 

In [359]:
df=pd.read_csv("houses.csv") #reads the houses.csv file, which is in the same directory as this file
l=len(df) #gets the amount of rows

#Goes through each row to check for null values
for i in range(0,l):
    data=df.loc[i] #puts the specific row into an item called "data"
    ll=len(data) #gets the length of a row
    
    #Checks each value in the row to see if it is null
    for ii in range(0,ll):
        if math.isnan(data[ii]):
            df.drop(i,inplace=True) #drops the row with a null value
            
#Used the pandas documentation to try and reindex this and get the inplace above because it was not working otherwise because otherwise index 14 does not exist
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
df.reset_index(inplace=True,drop=True) #Resets the index for the dropped places
df.head(16)

,age,area,bathrooms,distance metro,distance cemetery,cats,howls,clouds,precipitation,misery index,ice cream sold,haunted
0,65.06,2041,1.0,7.1,10.01,7,3,1.00,0.82,12.99,273,-0.596150
1,141.48,1564,0.0,7.4,4.07,5,5,1.00,0.99,16.77,184,-0.146465
2,7.58,1637,3.0,7.0,3.36,2,0,1.00,1.17,16.49,141,-0.303117
3,51.47,2021,2.0,7.9,3.43,6,8,0.13,0.92,8.28,146,0.339912
4,259.44,1642,1.0,7.5,3.19,4,1,1.00,1.73,5.90,178,0.724867
5,287.93,2676,2.0,8.1,4.57,7,6,0.89,1.37,8.59,186,0.805338
6,86.92,2494,0.0,7.1,4.69,4,4,1.00,0.90,16.55,259,-0.341216
7,184.49,1157,3.0,6.8,3.13,6,4,0.40,0.88,6.07,105,0.182997
8,42.52,2520,1.0,6.1,3.42,3,3,1.00,0.90,6.79,317,0.301654
9,150.29,1790,1.0,5.6,4.44,4,9,1.00,0.91,7.32,285,0.152310


**Part B**: Perform the appropriate statistical test at the $\alpha = 0.01$ significance level to determine if _at least one_ of the features is related to the the response $y$.  Clearly describe your methodology and show all computations in Python. 

This will be following Notebook 22, using ols to get the $\beta$s

As for the test:

$H_0$: $\beta$=0 for all $\beta$

$H_1$: $\beta$!=0 for at least one $\beta$

SST=$\sum_{i=1}^{n} (y_i - \bar{y})^2$

SSE=$\sum_{i=1}^{n} (y_i - \hat{y})^2$ = $\sum_{i=1}^{n} (y_i - (\hat{\beta_0}+\hat{\beta_1} x_1...+\hat{\beta_p} x_p))^2$

F=$\frac{\frac{SST-SSE}{p}}{\frac{SSE}{n-p-1}}$

Methodology:

Get the F statistic (using equations above)

Convert the F statistic to the P-value

Compare the p-value to the significance value for the hypothesis test

In [360]:
#-----------------------------Model-Variable-Setup----------------------------
important=df[["age","area","bathrooms","distance metro","distance cemetery","cats","howls","clouds","precipitation","misery index","ice cream sold"]]
x=sm.add_constant(important)
y=df[["haunted"]]

model=sm.OLS(y,x).fit() #Drawing of the model, giving us the betas as params

ybar=df["haunted"].mean() #Y-bar to calculate the SST

SST=0 #initializer of sst to be summed up 
SSE=0 #initializer of sse to be summed up

n=len(df) #number of samples, as the length of the dataframe
p=len(model.params) #number of groups, calculated from the params (the 'haunted' is taken out and a beta_0 is added, so the groups are the same number)

#For loop to go through the data and sum up the sst and sse
for i in range(0,n):
    yhat=0 #holder for the sum of all given values per sample
    data=df.loc[i] #data holder for a specific trial
    SST+=(data[p-1]-ybar)**2 #pulls the haunted value to make the sst for each loop
    
    #for loop to go through each part of the sample and bring in the betas to get the sse
    for ii in range(0,p):
        if ii!=0: #checks for all values that are not the constant beta
            yhat+=model.params[ii]*data[ii-1] #mulitplies the non-constant beta by the xi for the sse
        else:
            yhat+=model.params[ii] #add in the constant
    
    SSE+=(data[ll-1]-yhat)**2 #actual computation of the SSE for each loop
    
num=(SST-SSE)/p #numerator for the f-statistic equation
denom=SSE/(n-p-1) #denominator for the f-statistic
f=num/denom #final f statistic

pv=1-stats.f.cdf(f,p,n-p-1) #turning the f-statistic into a p-value

if pv<0.01: #Checks if the p-value is less than the significant value. if so, there is at least one non-zero beta
    print("There is at least one significant value")
else:
    print("There is no significant beta value")

There is at least one significant value


 **Part C**: Write a function `forward_select(df, resp_str, maxk)` that takes in the DataFrame, the name of the column corresponding to the response, and the maximum number of desired features, and returns a list of feature names corresponding to the `maxk` most important features via forward selection.  At each stage in forward selection you should add the feature whose inclusion in the model would result in the lowest sum of squared errors $(SSE)$. Use your function to determine the best $k=5$ features to include in the model. Clearly indicate which feature was added in each stage. 

**Note**: The point of this exercise is to see if you can implement **foward_select** yourself.  You may of course use canned routines like statmodels OLS, but you may not call any Python method that explicitly performs forward selection.

SSE=$\sum_{i=1}^{n} (y_i - \hat{y})^2$ = $\sum_{i=1}^{n} (y_i - (\hat{\beta_0}+\hat{\beta_1} x_1...+\hat{\beta_p} x_p))^2$

In [361]:
#A function to make the model, making it contained and easier to change if we want a different y parameter, etc later
def make_model(df,res):
    columns=np.array([df.columns.values]) #column names obtained from the dataframe
    important=columns[columns!=res] #cancel out the one that is the response (y value)
    imWithDf=df[important] #drawing values with those names (named after important just with the df because i forgot about it)
    
    #Model definitions
    x=sm.add_constant(imWithDf)
    y=df[[res]]

    model=sm.OLS(y,x).fit() #Drawing of the model, giving us the betas as params
    return model,important

#Handles the Single Linear regression step of the forward select
def slr(df,res,notY,model):
    n=len(df) #the number of samples
    p=len(notY) #The number of groups
    haunted=df[res] #a list of the haunted values to use as yi
    const=model.params[0] #The constant value of the relation
    
    feature="" #Holder for the most significant feature
    sse=1000 #Holder for the sse obtained, set to an absurd value to guarentee the first will snag it
    
    #For loop to compute sse's for every feature and display the most significant at the end
    for i in range(0,p):
        holder=0 #a holder for the sse of this feature
        data=df[notY[i]] #a holder for the specific column
        
        #For loop to sum the sse
        for ii in range(0,n):
            newData=data.loc[ii] #Specific value xi
            thisHaunted=haunted.loc[ii] #Specific value yi
            holder+=(thisHaunted-(const+model.params[i+1]*newData))**2 #Computation of the sse given xi and yi
        
        if holder<sse: #Checks if this specific sse is the lowest
            sse=holder #Sets the new lowest sse
            feature=notY[i] #Holds the feature that is the lowest sse
    return feature

#A function to handle the p-1 multilinear calculations needed for the forward select
def mlr(df,res,notY,model,arr):
    n=len(df) #number for rows
    p=len(notY) #number of columns
    haunted=df[res] #holder for the values of the y value
    const=model.params[0] #The constant value of the relation
    
    feature="" #holder for the eventual decision of the feature
    sse=1000 #holder for the best sse
    e=0 #holder for the index value of the resulting item
    b=0 #holder for the beta of the resulting item
    
    larr=len(arr) #length of the number we already went through
    beta=np.zeros(larr) #holder of the betas of the ones we already counted
    index=np.zeros(larr,dtype=int) #index values for the ones we have already taken into account
    
    count=0 #a counter that determines where in the arrays the values need to go
    
    #A while loop to continuously loop until we have filled all already-determined values
    while count!=larr:
        #a for loop to look for the final locations of already counted values
        for i in range(0,p):
            if notY[i]==arr[count]: #a check to see if this is the previously used column we are searching for
                beta[count]=model.params[i+1] #Holder of the beta value for this specific instance
                index[count]=i #Holder of the index of this specific instance
                count+=1 #up the count so we can put the next set of values into the next slot of the arrays
                break #end the current iteration, as we found where this value is
            
    #For loop to compare sse's to get the most optimal one
    for i in range(0,p):
        inde=index[index==i] #checks if this index is in the list of used indecies
        
        if len(inde)>0: #checks if this is indeed an index already used
            continue #skip to the next column if we used this before
            
        holder=0 #a value to hold the current sse for later comparison
        data=df[notY[i]] #holder of the current column
        
        #A for loop to calculate the sse of the current column by going through its xi's
        for ii in range(0,n):
            c=0 #a value to hold the summation just of the values we calculated previously
            newData=data.loc[ii] #current column's xi value
            thisHaunted=haunted.loc[ii] #the current yi for the given y
            
            #A for loop to simply add the portion of the relationship whose values we aready know best fit this
            for ccc in range(0,larr):
                c+=beta[ccc]*df[arr[ccc]].loc[ii] #calculation of beta_k x_k for values we have decided already fit best
                
            holder+=(thisHaunted-(const+c+model.params[i+1]*newData))**2 #sse for this column
            
        if holder<sse: #checks if this is the lowest sse we have calculated
            sse=holder #the lower sse now becomes the new sse value
            e=i #the index holder is now holding the index of this feature
            b=model.params[i+1] #the beta value, given by the param at i+1 to offset for the constant
            feature=notY[i] #the current best feature is recorded
    
    indexWin=np.array([e,b]) #puts the best index and the beta as an array so we can append it to the index and beta lists
    index=np.append(index,indexWin[0]) #appending the index of the new value to the previous list
    beta=np.append(beta,indexWin[1]) #appending the beta of the new value to the previous list
    return feature,beta,index

def forward_select(df,resp_str,maxk):
    model,notY=make_model(df,resp_str) #creates the model
    feature=slr(df,resp_str,notY,model) #gets the first feature from a simple linear relationship
    theFeatures=np.array([feature]) #starts an array of the features
    print("The first feature to be kept is {}".format(feature)) #indicates the first added feature
    
    #for loop to get the rest of the features
    for i in range(1,maxk):
        feature,beta,index=mlr(df,resp_str,notY,model,theFeatures) #gets the next required feature
        print("The next feature to be kept is {}".format(feature)) #prints that feature, as in the instructions "Clearly indicate which feature was added in each stage."
        nextFeature=np.array([feature]) #put the new feature in an array so it can be appended
        theFeatures=np.append(theFeatures,nextFeature) #appends the feature to the feature list
        
    #Print values to be used in part D  
    print(" ")
    print("The betas are {} with a constant {}".format(beta,model.params[0]))
    print("The beta subscript (location) of these values is {}".format(index))
    return theFeatures

In [362]:
features=forward_select(df,"haunted",5)
print("The features list returned from the forward_select method is {}".format(features))

The first feature to be kept is distance cemetery
The next feature to be kept is age
The next feature to be kept is precipitation
The next feature to be kept is bathrooms
The next feature to be kept is howls
 
The betas are [-0.09995715  0.00173249  0.00684016 -0.04109466  0.00564369] with a constant 0.3479565106233795
The beta subscript (location) of these values is [4. 0. 8. 2. 6.]
The features list returned from the forward_select method is ['distance cemetery' 'age' 'precipitation' 'bathrooms' 'howls']


**Part D**: Write down the multiple linear regression model, including estimated parameters, obtained by your forward selection process. 

$y=\beta_0 + \beta_5 x_5 + \beta_1 x_1 + \beta_9 x_9 + \beta_3 x_3 + \beta_7 x_7$  < 1 added to each index to offset for $\beta_0$ >

$y=0.3480 - 0.1000x_5 + 0.0017 x_1 + 0.0068 x_9 - 0.0411 x_3 + 0.0056 x_7$

**Part E**: Perform the appropriate statistical test at the $\alpha = 0.05$ significance level to determine whether there is a statistically significant difference between the full model with all features and the reduced model obtained by forward selection in **Part D**. 

$H_0$: All unused $\beta$=0

$H_1$: At least one unused $\beta$ != 0

n=number of instances

p=number of groups/columns

k=number of used terms in the reduced models (5 in this case)

F=$\frac{\frac{(SSE_{reduced}-SSE_{full})}{p-k}}{\frac{SSE_{full}}{n-p-1}}$

In [363]:
#a modified version of the forward select from before so I do not have to have all those annoying print statements
def forward_select_modified(df,resp_str,maxk):
    model,notY=make_model(df,resp_str) #creates the model
    feature=slr(df,resp_str,notY,model) #gets the first feature from a simple linear relationship
    theFeatures=np.array([feature]) #starts an array of the features
    
    #for loop to get the rest of the features
    for i in range(1,maxk):
        feature,beta,index=mlr(df,resp_str,notY,model,theFeatures) #gets the next required feature
        nextFeature=np.array([feature]) #put the new feature in an array so it can be appended
        theFeatures=np.append(theFeatures,nextFeature) #appends the feature to the feature list
        
    return theFeatures

#models and features pulled from part c; if you wish to run this one, please run part c first
model,notY=make_model(df,"haunted")
features=forward_select_modified(df,"haunted",5)

n=len(df) #The amount of samples tested
p=len(notY) #The amount of columns that are not haunted (or another value)
k=5 #The max k from before, which is the k in this case

#A function to calculate then return the SSE of the reduced and full multilinear relationships
def get_sse(df,res,model,arr):
    larr=len(arr) #The amount of features
    count=0 #a count variable to get the betas and indexes for the chosen features
    
    beta=np.zeros(larr) #holder of the betas of the ones we already counted
    index=np.zeros(larr,dtype=int) #index values for the ones we have already taken into account
    
    #A while loop to continuously loop for the index and betas for the values. Brought in from part c for a similar purpose
    while count!=larr:
        #a for loop to look for the final locations of already counted values
        for i in range(0,p):
            if notY[i]==arr[count]: #a check to see if this is the previously used column we are searching for
                beta[count]=model.params[i+1] #Holder of the beta value for this specific instance
                index[count]=i #Holder of the index of this specific instance
                count+=1 #up the count so we can put the next set of values into the next slot of the arrays
                break #end the current iteration, as we found where this value is
                
    sseFull=0 #holder for the full SSE
    sseRed=0 #holder for the SSE of the reduced model
    const=model.params[0] #The constant value
    
    #A for loop to go through the sample's values and calculate their SSE's
    for i in range(0,n):
        data=df.loc[i] #Holder for the data for one specific instance
        haunted=df[res].loc[i] #This loop's value for haunted (or another instance)
        holder=0 #Holder of this loop's SSE for the full SSE
        holderRed=0 #Holder of this loop's SSE for the reduced SSE
        
        #For loop to compute the SSE for this loop's full SSE
        for ii in range(0,p):
            holder+=data[ii]*model.params[ii+1] #beta_1 x_1 + beta_2 x_2...
            
        sseFull+=(haunted-(holder+const))**2 #calculation of the full SSE
        
        #For loop to compute the SSE for the specific featured values
        for ii in range(0,larr):
            e=notY[index[ii]] #Taking the specific column we need to draw
            c=df[e].loc[i] #Getting the value of that column at the ith row
            holderRed+=beta[ii]*c #beta_k x_k
            
        sseRed+=(haunted-(holderRed+const))**2 #Calculation of the reduced model's SSE
        
    return sseFull,sseRed
        
sseFull,sseRed=get_sse(df,"haunted",model,features) #Call to calculate the SSE's

num=(sseRed-sseFull)/(p-k) #Numerator to the f statistic
denom=sseFull/(n-p-1) #Denominator to the f statistic
f=num/denom #getting the f
p=1-stats.f.cdf(f,p-k,n-p-1) #Obtaining the p value to test if all is fine and dandy

if p<=0.05: #Test if there is a significant difference between the two models
    print("There is a statistically significant difference between the two models")
    print("The p-value {} is less than the significance value 0.05".format(p))
else:
    print("There is NO statistically significant difference between the two models")
    print("The p-value {} is greater than the significance value 0.05".format(p))

There is a statistically significant difference between the two models
The p-value 0.014950622360453636 is less than the significance value 0.05


**Part F**: Based on your conclusions in **Part E**, use the _better_ of the two models to predict the haunted house hazard when the following features are observed: 

- $\texttt{age}$: 100 years
- $\texttt{area}$: 2200 square feet
- $\texttt{bathrooms}$: 3 bathrooms
- $\texttt{distance metro}$: 25 miles
- $\texttt{distance cemetery}$: 10 miles
- $\texttt{cats}$: 4 cats
- $\texttt{howls}$: 5 wolf howls/night
- $\texttt{clouds}$: 0.65 cloud cover
- $\texttt{precipitation}$: 0 inches
- $\texttt{misery index}$: 10
- $\texttt{ice cream sold}$: 125

In [364]:
#Creation of the model using the function defined in part C
model,notY=make_model(df,"haunted")

newValues=np.array([100,2200,3,25,10,4,5,0.65,0,10,125]) #Array of the values given above

l=len(newValues) #The amount of values to test
summ=0 #summator to get the haunted value of y

#For loop to sum the xi multiplied by their betas
for i in range(0,l):
    summ+=newValues[i]*model.params[i+1] #beta_k x_k
    
haunted=summ+model.params[0] #Adding the constant value to get the haunted value

print("The haunted value is {}".format(haunted)) #print the haunted value

The haunted value is -0.7782125496062187


**Part G:** Governor Polis dabbles a bit in the art of data science, as well as the science of data art. He tells you that the response (`haunted` score) that you and Amy predicted is actually the natural logarithm of the _odds_ that a house with the given features is haunted, where if $p$ is the probability that a house is haunted, then the odds are given by $$\text{odds} = \dfrac{p}{1-p}$$

What transformation must you make in order to turn your multiple regression model into a logistic regression model, to classify a house a haunted or not? Perform this simple transformation, then use a decision threshold of 0.5 to classify the house from **Part F** as haunted or not haunted. No new models should be fit here; use the same model that you used in Part F.

p is the probability we need

odds=$\frac{p}{1-p}$

odds(1-p)=p

odds-podds=p

$\frac{odds}{p}$ - odds=1

$\frac{odds}{p}$=1+odds

$\frac{p}{odds}$=$\frac{1}{1+odds}$

p=$\frac{odds}{1+odds}$

odds=$e^{\beta_0+\beta_1 x_1+...+\beta_p x_p}$

In [365]:
model,notY=make_model(df,"haunted") #Creation of the model so we can use the betas
p=len(newValues) #number of fields to test
const=model.params[0] #the constant value so we can use it later
summ=0 #a summation variable to get the multilinear equation

newValues=np.array([100,2200,3,25,10,4,5,0.65,0,10,125]) #Array of the values given in the previous part

#For loop to get the values of the equation with the given betas
for ii in range(0,p):
    summ+=newValues[ii]*model.params[ii+1] #beta_k x_k
    
odds=math.exp(const+summ) #calculation of odds using the odds equation
pw=odds/(1+odds) #Getting the probability with the transformation calculated above
ps=1/(1+math.exp(-(const+summ))) #getting the probability with the sigmoid function as a sanity check

print("The probability is {}".format(pw)) #Print the probability gotten from the odds conversion
print("As a sanity check, the probability is {} using the sigmoid equation instead".format(ps)) #sanity check with the sigmoid function

if pw>=0.5: #checks if the value is above or below our 0.5 threshold
    print("This probability is greater than the threshold of 0.5, so it is likely haunted")
else:
    print("This probability is lower than the threshold of 0.5, so it is likely not haunted")

The probability is 0.3147052503432849
As a sanity check, the probability is 0.3147052503432849 using the sigmoid equation instead
This probability is lower than the threshold of 0.5, so it is likely not haunted


<br>

---
<a id='p2'></a>
[Back to top](#top)

### [30 points] Problem 2: Definitions check 


Let $Z \sim N(0, 1)$ and define $W = Z^2$. It is clear that $W$ and $Z$ are not
independent (clearly $W$ depends on $Z$).

Show that $Cov(W, Z) = 0$ even though $W$ and $Z$ are
dependent.


Cov(W,Z)=E[(W-E[W]) * (Z-E[Z])]

Cov(W,Z)=E[($Z^2$-E[$Z^2$]) * (Z-E[Z])]

E[x]=$\int_{-\infty}^{\infty} x f(x) dx$

Cov(W,Z)=E[($Z^2$-$\int_{-\infty}^{\infty} z(z^2) dz$) * (Z-$\int_{-\infty}^{\infty} z(z) dz$)]

---

$\int_{-\infty}^{\infty} z^3 dz$

$\frac{1}{4} z^4 | -\infty to \infty$

$\infty - \infty$

(Realizes $\infty - \infty$ is not 0)

---

$\int_{-\infty}^{\infty} z^2 dz$

$\frac{1}{3} z^3 | -\infty to \infty$

$\infty - (-\infty)$

$\infty + \infty$

$\infty$

---

Cov(W,Z)=E[($Z^2-(\infty-\infty)$) * (Z-$\infty$)]

Cov(W,Z)=E[($Z^2-(\infty-\infty)$) * (-$\infty$)]

($\infty-\infty$ is just some constant, thus can still be absorbed with $Z^2$ into the -$\infty$)

Cov(W,Z)=E[(-$\infty)$]

Cov(W,Z)=$\int_{-\infty}^{\infty} -\infty$ dz

Cov(W,Z)=0 | $-\infty to \infty$

Cov(W,Z)=0

<br>

---
[Back to top](#top)
<a id='bot'></a>